## Load the model

In [ ]:
import rpad.pyg.nets.pointnet2 as pnp_orig
from open_anything_diffusion.models.modules.dit_models import DGDiT, DiT, PN2DiT, PN2HisDiT
from open_anything_diffusion.models.modules.history_encoder import HistoryEncoder
from open_anything_diffusion.models.flow_trajectory_predictor import FlowTrajectoryInferenceModule
from open_anything_diffusion.models.flow_diffuser_dit import FlowTrajectoryDiffuserInferenceModule_DiT
from open_anything_diffusion.models.flow_diffuser_dgdit import FlowTrajectoryDiffuserInferenceModule_DGDiT
from open_anything_diffusion.models.flow_diffuser_hispndit import FlowTrajectoryDiffuserInferenceModule_HisPNDiT
# from open_anything_diffusion.models.flow_diffuser_pndit import FlowTrajectoryDiffuserInferenceModule_PNDiT
inference_module_class = {
    "dit": FlowTrajectoryDiffuserInferenceModule_DiT,
    "dgdit": FlowTrajectoryDiffuserInferenceModule_DGDiT,
    "hispndit": FlowTrajectoryDiffuserInferenceModule_HisPNDiT,
    "flowbot": FlowTrajectoryInferenceModule
}
networks = {
    "flowbot": pnp_orig.PN2Dense(
        in_channels=0,
        out_channels=3,
        p=pnp_orig.PN2DenseParams(),
    ),
    "dit": DiT(in_channels=6, depth=5, hidden_size=128, num_heads=4, learn_sigma=True).cuda(),
    "dgdit": DGDiT(in_channels=3, depth=5, hidden_size=128, patch_size=1, num_heads=4, n_points=1200).cuda(),
    "hispndit": {
        "DiT": PN2HisDiT(
            history_embed_dim=128,
            in_channels=3,
            depth=5,
            hidden_size=128,
            num_heads=4,
            learn_sigma=True,
        ).cuda(),
        "History": HistoryEncoder(
            history_dim=128,
            history_len=1,
            batch_norm=True,
            transformer=False,
            repeat_dim=False,
        ).cuda(),
    }
    # "pndit": PN2DiT(in_channels=3, depth=5, hidden_size=128, patch_size=1, num_heads=4, n_points=1200),  
}

In [ ]:
class InferenceConfig:
    def __init__(self):
        self.batch_size = 1
        self.trajectory_len = 1
        self.mask_input_channel = False

inference_config = InferenceConfig()

class ModelConfig:
    def __init__(self):
        self.num_train_timesteps = 100

model_config = ModelConfig()

In [ ]:
import os
ckpt_dir = './pretrained'
train_type = 'fullset_half_half'   # door_half_half, fullset_half_half - what dataset the model is trained on 
model_type = 'hispndit'   # dit, dgdit, hispndit - model structure
ckpt_path = os.path.join(ckpt_dir, f'{train_type}_{model_type}.ckpt')

In [ ]:
model = inference_module_class[model_type](
    networks[model_type], inference_cfg=inference_config, model_cfg=model_config
)
model.load_from_ckpt(ckpt_path)
model.eval()
model.cuda()

## Make a prediction

Read the point cloud

In [ ]:
import numpy as np
pcd_dir = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu'
pcd_paths = [os.path.join(pcd_dir, pcd_name) for pcd_name in os.listdir(pcd_dir)]

In [ ]:
id = 2
# path = pcd_paths[id]
# path = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu/incorrect_toilet.npy'
# path = '/home/yishu/Azure_Kinect_ROS_Driver/src/pc_data_for_yishu/fridge_L_open_45.npy'
path = '/home/yishu/flowbot_panda/failure_case_with_additional_crop_world.npy'
print(path)
pcd = np.load(path) * 3

Rotate the point cloud

In [ ]:
import numpy as np
rot = np.array([[0, -1, 0], [1,  0, 0], [0,  0, 1]])
mean_x = pcd[:, 0].mean()
mean_y = pcd[:, 1].mean()
rot_pcd = pcd.copy()
rot_pcd[:, 0] -= mean_x
rot_pcd[:, 1] -= mean_y
# rot_pcd[:, 2] += 1
rot_pcd = rot_pcd@rot.T

Sample it to 1200 points

In [ ]:
# Could use pytorch3d for this but it has some cuda conflict with my current env and I don't wnana change lol
import numpy as np

def farthest_point_sampling(world_points, points, k):
    num_points = points.shape[0]
    chosen_indices = np.zeros(k, dtype=int)
    chosen_indices[0] = np.random.randint(num_points)
    distances = np.full(num_points, np.inf)
    
    for i in range(1, k):
        dist = np.linalg.norm(points - points[chosen_indices[i-1]], axis=1)
        distances = np.minimum(distances, dist)
        chosen_indices[i] = np.argmax(distances)
        
    return points[chosen_indices], world_points[chosen_indices]

# Example usage
# sampled_points = farthest_point_sampling(pcd, 1200)
sampled_points, sampled_points_world = farthest_point_sampling(pcd, rot_pcd, 1200)

print(sampled_points.shape)

- If don't use history

In [ ]:
# Random sampling
# import torch
# sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
pred_flow = model.predict(sampled_points)[:, 0, :]

- If use history

In [ ]:
history_pcd = np.zeros_like(sampled_points)
history_flow = np.zeros_like(sampled_points)
pred_flow = model.predict(sampled_points, history_pcd=history_flow, history_flow=history_pcd)[:, 0, :]

In [ ]:
# import tqdm
# from open_anything_diffusion.metrics.trajectory import flow_metrics
# gt_flow = torch.zeros(1200, 3)
# gt_flow[:, 2] = 1 
# cosines = []
# for i in tqdm.tqdm(range(20)):
#     sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
#     pred_flow = model.predict(sampled_points)[:, 0, :]
#     rmse, cos_dist, mag_error = flow_metrics(
#         pred_flow, gt_flow, reduce=True
#     )
#     cosines.append(cos_dist)

# print(max(cosines))

## Visualize the prediction

In [ ]:
pred_flow.shape

In [ ]:
import torch
# sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
gt_action = torch.tensor([-1, 0, 0])
cosines = []
predictions = []
for i in range(50):
    pred_flow = model.predict(sampled_points)[:, 0, :]
    idx = torch.topk(pred_flow, k=1)[1][0]
    action = pred_flow[idx]
    cosine = torch.cosine_similarity(action, gt_action)
    cosines.append(cosine)
    predictions.append(pred_flow)

In [ ]:
cosines

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(sampled_points),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([sampled_points]),
    # torch.as_tensor([pred_flow.cpu().numpy() * 2]),
    torch.as_tensor([predictions[6].cpu().numpy()]* 5),
    "red",
)
fig = animation.animate()
fig.show()

### The goal point and orientation

In [ ]:
from scipy.spatial.transform import Rotation 
def get_contact_point_and_flow_vector(flow, xyz):
    magnitude = torch.norm(flow, dim=1)
    idx_of_max_flow = torch.argmax(magnitude.unsqueeze(1))
    contact_point = torch.from_numpy(xyz[idx_of_max_flow])
    flow_vector = flow[idx_of_max_flow] 
    flow_vector_normalized = (flow_vector / flow_vector.norm(dim=-1)).float()
    return contact_point, flow_vector_normalized


def get_goal_point_and_orientation(contact_point, flow_vector):
    goal_point = contact_point + 0.2 * flow_vector
    e_z_init = torch.tensor([0, 0, 1.0]).float().cuda()
    e_y = -flow_vector
    e_x = torch.linalg.cross(e_y, e_z_init)
    e_x = e_x / e_x.norm(dim=-1)
    e_z = torch.linalg.cross(e_x, e_y)
    R_goal = torch.stack([e_x, e_y, e_z], dim=1).cuda()
    R_gripper = torch.as_tensor(
        [
            [1, 0, 0],
            [0, 0, 1.0],
            [0, -1.0, 0],
        ]
    ).cuda()

    goal_orientation_mat = (R_goal @ R_gripper).cpu()
    goal_orientation_quat = Rotation.from_matrix((R_goal @ R_gripper).cpu()).as_quat()
    return goal_point, goal_orientation_quat, goal_orientation_mat


def transform_flow_contact_point_goal_point_and_orientation_to_world(contact_point, goal_point, goal_orientation, mean_x, mean_y, flow_vector):
    # Formatting goal_point and goal_orientation to be in the same frame as the point cloud so that it can be visualized
    R = torch.tensor([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]).float().cuda()

    goal_point = goal_point @ R.T
    contact_point = contact_point @ R.T

    # Add the mean in x and y
    goal_point[0] += mean_x
    goal_point[1] += mean_y
    contact_point[0] += mean_x
    contact_point[1] += mean_y
    
    goal_point = goal_point.cpu().numpy()
    goal_point = np.reshape(goal_point, (1, 3))
    contact_point = contact_point.cpu().numpy()
    contact_point = np.reshape(contact_point, (1, 3))

    goal_orientation = Rotation.from_quat(goal_orientation)
    goal_orientation = torch.from_numpy(goal_orientation.as_matrix()).float().cuda()
    goal_orientation = R @ goal_orientation
    goal_orientation = Rotation.from_matrix(goal_orientation.cpu()).as_quat()

    flow_vector = flow_vector @ R.T

    return contact_point, goal_point, goal_orientation, flow_vector

In [ ]:
import torch
import numpy as np
contact_point, flow_vector_normalized = get_contact_point_and_flow_vector(pred_flow, sampled_points)
contact_point = contact_point.float().cuda()
flow_vector_normalized = flow_vector_normalized.float().cuda()
goal_point, goal_orientation_quat, goal_orientation_mat = get_goal_point_and_orientation(contact_point, flow_vector_normalized)
# contact_point_world, goal_point_world, goal_orientation_world,flow_vector = transform_flow_contact_point_goal_point_and_orientation_to_world(contact_point, goal_point, goal_orientation_quat, mean_x, mean_y, flow_vector_normalized)

In [ ]:
flow_vector_normalized

In [ ]:
print(contact_point)
print(goal_point)
print(goal_orientation_quat)
print(goal_orientation_mat)

In [ ]:
# import copy
# import open3d as o3d
# goal_orientation_frame = (o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=goal_point[0]))
# goal_orientation_frame_r = copy.deepcopy(goal_orientation_frame)
# rot = goal_orientation_frame.get_rotation_matrix_from_quaternion(goal_orientation_quat)
# goal_orientation_frame_r.rotate(rot, center=goal_point[0])
# mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])

# pc = o3d.geometry.PointCloud()
# pc.points = o3d.utility.Vector3dVector(sampled_points)  # Random points

# # Create a visualizer object and add the point cloud
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.add_geometry(goal_orientation_frame_r)
# vis.add_geometry(pc)
# vis.run()
# vis.destroy_window()

#### Flowbot frame

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation

magnitude = torch.norm(pred_flow, dim=1)
idx_of_max_flow = torch.argmax(magnitude)
# print(idx_of_max_flow, pred_flow[idx_of_max_flow])
masked_pred_flow = pred_flow.clone() * 0.1
masked_pred_flow[idx_of_max_flow, :] *= 20
print(idx_of_max_flow, masked_pred_flow[idx_of_max_flow], goal_orientation_mat)

animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(sampled_points),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([sampled_points]),
    torch.as_tensor([masked_pred_flow.numpy()]),
    "red",
)
animation.add_trace(
    torch.as_tensor(goal_point.unsqueeze(0).repeat(3, 1).cpu().numpy()),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([goal_point.unsqueeze(0).repeat(3, 1).cpu().numpy()]),
    torch.as_tensor([goal_orientation_mat.T.cpu().numpy()]),
    "yellow",
)

fig = animation.animate()
fig.show()

In [ ]:
print(contact_point)
print(goal_point)
print(goal_orientation_mat)

#### World frame

In [ ]:
contact_point_world, goal_point_world, goal_orientation_world, flow_vector = transform_flow_contact_point_goal_point_and_orientation_to_world(contact_point, goal_point, goal_orientation_quat, mean_x, mean_y, flow_vector_normalized)
goal_orientation_mat_world = Rotation.from_quat(goal_orientation_world).as_matrix()

In [ ]:
print(contact_point_world)
print(goal_point_world)
print(goal_orientation_mat_world)

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation

# magnitude = torch.norm(pred_flow, dim=1)
# idx_of_max_flow = torch.argmax(magnitude)
masked_pred_flow = pred_flow.clone() * 0.1
masked_pred_flow[idx_of_max_flow, :] *= 20

R = torch.tensor([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]).float()

masked_pred_flow_world = masked_pred_flow @ R.T
print(idx_of_max_flow, masked_pred_flow_world[idx_of_max_flow], goal_orientation_mat_world)

animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(sampled_points_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([sampled_points_world]),
    torch.as_tensor([masked_pred_flow_world.numpy()]),
    "red",
)
animation.add_trace(
    torch.as_tensor(torch.from_numpy(goal_point_world).repeat(3, 1).cpu().numpy()),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([torch.from_numpy(goal_point_world).repeat(3, 1).cpu().numpy()]),
    torch.as_tensor([goal_orientation_mat_world.T]),
    "yellow",
)

fig = animation.animate()
fig.show()

In [ ]:
import plotly.graph_objects as go

# Define vectors
vectors = [
    dict(x=[0, 3], y=[0, 2], z=[0, 1], name='Vector 1'),
    dict(x=[0, 1], y=[0, 2], z=[0, 3], name='Vector 2'),
    dict(x=[0, 1], y=[0, 0], z=[0, 1], name='Vector 3')
]

# Create the figure
fig = go.Figure()

# Add vectors to plot
for v in vectors:
    fig.add_trace(go.Scatter3d(x=v['x'], y=v['y'], z=v['z'], mode='lines+markers+text', name=v['name']))

# Update layout for a nice aspect ratio
fig.update_layout(scene=dict(
    xaxis=dict(nticks=4, range=[-5,5]),
    yaxis=dict(nticks=4, range=[-5,5]),
    zaxis=dict(nticks=4, range=[-5,5])
), width=700)

# Show the plot
fig.show()


In [ ]:
print(contact_point)
print(goal_point, goal_orientation)
print(contact_point, goal_point, goal_orientation)

In [ ]:
print(flow_vector_normalized)

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([pcd]),
    torch.as_tensor([np.zeros_like(pcd)]),
    "red",
)
fig = animation.animate()
fig.show()

## About the policy

- Switch grasp point

In [ ]:
# Pseudo codes
def switch_grasp_point(last_gripper_pos, current_gripper_pos, flow_prediction, current_pcd):
    # 1 - find the point in current_pcd closest to current_grasp_point
    grasp_point_id = 0  # current_pcd's closest point id
    grasp_flow = flow_prediction[grasp_point_id]
    # 2 - Compare the grasp point flow with the max prediction flow
    leverage_increase = flow_prediction.norm(dim=-1).max() - grasp_flow.norm()
    if last_gripper_pos - current_gripper_pos < 0.01 or leverage_increase > 0.2:  # move threshold
        return True
    return False

- Use / not use history

Basically we want to have a signal that tells us whether the last step makes positive progress

Policy: Maintain a correct direction stack

For each step, record the movement of the gripper (delta gripper)

1) When the stack is empty, only push the delta gripper in the stack when the |delta gripper| > threshold (means that the gripper successfully moves the object)

2) Always compare the current predicted direction with the direction from the stack top: 

- if different by over 90 degree, don't push the current action in and also don't execute this step
- Execute the step and push the delta gripper of this step in the stack